## 같은 시간에 방송된 대표product를  exp_min, total, sales를 합쳐서 가져오기
* 같은 시간에 판매하며, 옵션이 다른 (예를 들면 여성용/남성용, 무이자/일시불, 10인용/6인용)도 하나로 합쳐서 각 각 노출시간 나오게 하기
* raw data에서 무형, total=0인 data drop 시키지 x

In [ ]:
# !pip install plotly
import pandas as pd
import numpy as np
import re
import os
import sys
import urllib.request
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm.notebook import tqdm

# pd.set_option('display.max_rows', 100)

In [ ]:
df = pd.read_excel('./bc/base_line_test.xlsx', header=0) # 무형, total_price가 0인 data를 삭제하지 않은 데이터 가져오기
df.p_name.loc[15751] = df.p_name.loc[15752]
# 위의 값은 '일시불 올리고 가스와이드그릴 프리미엄형'인데, 해당 방송에서 무이자만 exp_min을 가지고 있으므로 무이자로 바꿔준다.

### 같은 상품군에 있는 data를 같은 code로 나열하기

In [ ]:
# copy만들어 사용. 왜냐면 후에 df와 merge 해야하므로!
tmp_df= df.copy()

tmp_df['HOUR']= tmp_df['HOUR'].replace(0, 24) # 나중에 for문 - 할 때 , 필요한 부분! 23시 다음 연속된 방송은 0시이기 때문에 오류
tmp_df['HOUR']= tmp_df['HOUR'].replace(1, 25)
tmp_df['HOUR']= tmp_df['HOUR'].replace(2, 26)

tmp_df = tmp_df.dropna(subset=['exp_min']) #없는 값은 있는 값이랑 중복이기에 drop처리/ 있는값은 계속 있다는 idea

# 같은 날 같은 제품을 팔았을 경우 오류나서 hour 추가
df2 = tmp_df.groupby(['YEAR_DAY','p_group','m_code','p_code','p_name','HOUR']).agg({'exp_min':['sum'],'no':['min']})
df2.sort_index(ascending=False)

df2 = df2.groupby(level=[0,1,2,3,4,5]).sum()
df2.columns = ['exp_min','no'] #나중에 merge하기 편하게 변경
df2 = df2.reset_index() #multiindex라서 

df2['new_code'] = np.arange(0,len(df2.index)) #유일한 값 만들어주고, 번호 매기기

# hour가 다른 시간이지만, 같은 상품일 때! 그 전 상품이나 뒷 상품이 같은 제품일 경우는 code를 그 전의 값으로 가져옴
for idx in range(1, df2.shape[0]):
    if (df2.loc[idx, 'p_group'] == df2.loc[idx-1, 'p_group']) and (abs(df2.loc[idx, 'p_code'] - df2.loc[idx-1, 'p_code']) <= 1)\
        and (df2.loc[idx, 'p_name'] != df2.loc[idx-1, 'p_name']):
        df2.loc[idx, 'p_name'] = df2.loc[idx-1, 'p_name']
    
    if (df2.loc[idx, 'p_name'] == df2.loc[idx-1,'p_name']) and (abs(df2.loc[idx, 'HOUR'] - df2.loc[idx-1,'HOUR']) <= 1):
        df2.loc[idx, 'new_code'] = df2.loc[idx-1, 'new_code']

In [ ]:
# 같은 날 같은 제품을 팔았을 경우 오류나서 hour 추가
df2 = tmp_df.groupby(['YEAR_DAY','p_group','m_code','p_code','p_name','HOUR']).agg({'exp_min':['sum'],'no':['min']})
df2.sort_index(ascending=False)

df2 = df2.groupby(level=[0,1,2,3,4,5]).sum()
df2.columns = ['exp_min','no'] #나중에 merge하기 편하게 변경
df2 = df2.reset_index() #multiindex라서 index를 reset

df2['new_code'] = np.arange(0,len(df2.index)) #유일한 값 만들어주고, 번호 매기기

code_df= pd.merge(left=df, right=df2.iloc[:,[7,8]], how='left', on=['no']) #축소한 데이터와 원래 데이터 merge

code_df['new_code']= code_df['new_code'].fillna(method='ffill') #new_code 를 앞의 값으로 채우기
code_df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,no,new_code
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,53.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,0,13.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,110.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,1,13.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,82.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,2,13.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,3,13.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.0,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,0,4,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2019-12-31 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2019,12,31,0,20,0,2019-12-31,Tuesday,2,7187,9,38304,8734.0
38305,2019-12-31 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2019,12,31,0,40,0,2019-12-31,Tuesday,2,7187,9,38305,8734.0
38306,2019-12-31 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2019,12,31,1,0,0,2019-12-31,Tuesday,2,7187,9,38306,8735.0
38307,2019-12-31 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,2019,12,31,1,20,0,2019-12-31,Tuesday,2,7188,9,38307,8736.0


In [ ]:
# hour가 다른 시간이지만, 같은 상품일 때! 그 전 상품이나 뒷 상품이 같은 제품일 경우는 code를 그 전의 값으로 가져옴
for idx in range(1, df2.shape[0]):
    if (df2.loc[idx, 'p_group'] == df2.loc[idx-1, 'p_group']) and (abs(df2.loc[idx, 'p_code'] - df2.loc[idx-1, 'p_code']) <= 1)\
        and (df2.loc[idx, 'p_name'] != df2.loc[idx-1, 'p_name']):
        df2.loc[idx, 'p_name'] = df2.loc[idx-1, 'p_name']
    
    if (df2.loc[idx, 'p_name'] == df2.loc[idx-1,'p_name']) and (abs(df2.loc[idx, 'HOUR'] - df2.loc[idx-1,'HOUR']) <= 1):
        df2.loc[idx, 'new_code'] = df2.loc[idx-1, 'new_code']

In [ ]:
code_df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,p_group_code,no,new_code
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,53.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,0,13.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,110.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0,1,13.0
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,82.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,2,13.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.0,2019,1,1,6,20,0,2019-01-01,Tuesday,2,0,0,3,13.0
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.0,2019,1,1,6,40,0,2019-01-01,Tuesday,2,0,0,4,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2019-12-31 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2019,12,31,0,20,0,2019-12-31,Tuesday,2,7187,9,38304,8734.0
38305,2019-12-31 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2019,12,31,0,40,0,2019-12-31,Tuesday,2,7187,9,38305,8734.0
38306,2019-12-31 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,NaN,2019,12,31,1,0,0,2019-12-31,Tuesday,2,7187,9,38306,8735.0
38307,2019-12-31 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,NaN,2019,12,31,1,20,0,2019-12-31,Tuesday,2,7188,9,38307,8736.0


# ------------------------------------------------------------------------------------

## 데이터 sum하기

In [ ]:
tmp_df = code_df.copy()
df3 = tmp_df.groupby(['new_code']).agg({'exp_min':['sum'],'unit_price':['mean'], 'total_price':['sum'],'sales':['sum'],'no':['min']})
df3.columns = ['exp_min','unit_price','total_price','sales','no']
df3.reset_index()

sum_df= pd.merge(code_df.iloc[:,[0,2,3,4,5,9,10,11,12,13,14,15,16,17,19,20,21]],df3,on='no', how='right')
sum_df= sum_df[['broadcast','exp_min','m_code','p_code','p_name','p_group','unit_price','total_price','sales','YEAR','MONTH','DAY','HOUR','MINUTE','SECOND','YEAR_DAY','DAY_NAME','DAY_NUM','p_group_code','new_code','no']]

# sum_df

In [ ]:
# p_group == '무형'을 제거하기 위해서 그 리스트를 가져온다.

mh_list = []
for idx, value in enumerate(sum_df.p_group):
    if value == '무형':
        mh_list.append(idx)
len(mh_list)

530

In [ ]:
# sum_df[sum_df.p_group == '무형']
sum_df.drop(mh_list, inplace=True)
sum_df.reset_index(drop=True,inplace=True)

In [ ]:
sum_df

,broadcast,exp_min,m_code,p_code,p_name,p_group,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,p_group_code,new_code,no
0,2019-01-01 06:00:00,60.0,100346,201072,테이트 남성 셀린니트3종,의류,39900.0,32696000.0,820.0,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,13.0,0
1,2019-01-01 07:00:00,60.0,100305,200974,오모떼 레이스 파운데이션 브라,속옷,59000.0,47878000.0,812.0,2019,1,1,7,0,0,2019-01-01,Tuesday,2,1,7.0,6
2,2019-01-01 08:00:00,60.0,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,59900.0,99736000.0,1665.0,2019,1,1,8,0,0,2019-01-01,Tuesday,2,0,16.0,9
3,2019-01-01 09:00:00,60.0,100816,202404,보코 리버시블 무스탕,의류,79000.0,90973000.0,1151.0,2019,1,1,9,0,0,2019-01-01,Tuesday,2,0,18.0,12
4,2019-01-01 10:00:00,60.0,100809,202395,CERINI by PAT 남성 풀패키지 기모니트 3종,의류,79900.0,259678000.0,3250.0,2019,1,1,10,0,0,2019-01-01,Tuesday,2,0,17.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8226,2019-12-31 21:00:00,20.0,100501,201516,무이자 국내제조 노비타 뉴스파 비데 무료설치,생활용품,333500.0,102836000.0,413.0,2019,12,31,21,0,0,2019-12-31,Tuesday,2,6,8742.0,38279
8227,2019-12-31 21:20:00,40.0,100155,200556,일시불 LG전자 매직스페이스 냉장고,가전,1429000.0,117105000.0,82.0,2019,12,31,21,20,0,2019-12-31,Tuesday,2,5,8725.0,38283
8228,2019-12-31 22:00:00,20.0,100155,200556,일시불 LG전자 매직스페이스 냉장고,가전,1429000.0,104917000.0,75.0,2019,12,31,22,0,0,2019-12-31,Tuesday,2,5,8726.0,38287
8229,2019-12-31 23:20:00,40.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,163000.0,52902000.0,321.0,2019,12,31,23,20,0,2019-12-31,Tuesday,2,2,8757.0,38292


In [ ]:
# YEAR_DAY의 타입을 올바르게 수정
sum_df['YEAR_DAY'] = pd.to_datetime(sum_df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')

In [ ]:
# 연간 주번호 feature 추가
sum_df['broadcast'] = pd.to_datetime(sum_df['broadcast'], format='%Y-%m-%d %H:%M:%S', errors='raise')
sum_df['52_week'] = sum_df['broadcast'].dt.weekofyear
sum_df['52_week'][-35:] = 53

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# 브랜드 feature 추가
건강기능 =sum_df[sum_df['p_group'] == '건강기능']
생활용품 =sum_df[sum_df['p_group'] == '생활용품']
침구 =sum_df[sum_df['p_group'] == '침구']
주방 =sum_df[sum_df['p_group'] == '주방']
가전 =sum_df[sum_df['p_group'] == '가전']
가구 =sum_df[sum_df['p_group'] == '가구']
이미용 =sum_df[sum_df['p_group'] == '이미용']
농수축 =sum_df[sum_df['p_group'] == '농수축']
잡화 =sum_df[sum_df['p_group'] == '잡화']
속옷= sum_df[sum_df['p_group'] == '속옷']
의류=sum_df[sum_df['p_group'] == '의류']
p_group_name = ['의류', '속옷', '농수축', '이미용', '가전', '가구', '침구', '생활용품', '잡화', '건강기능', '주방']

for group in p_group_name :
    
    if group == '건강기능':
        p_name = 건강기능['p_name']
        
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('(일시불)|(무이자)|\((.*?)\)|(특집)|(단하루)','', name)
            brand_name = re.sub('[\[\]\"]', " ", str(brand_name))
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]

            brand_name = brand_name.replace('광동제약', '광동')
            brand_name = brand_name.replace('광동', '광동제약')

            brand_name = brand_name.replace('100%리얼착즙석류', '100%리얼')
            brand_name = brand_name.replace('100%리얼', '100%리얼착즙석류')

            brand_name = brand_name.replace('통째로', '통째로 착즙한 루비 석류즙 84포')
            brand_name = brand_name.replace('모나코사놀', '뉴트리원')
            brand_name = brand_name.replace('한삼인순홍삼진7박스', '한삼인')

            brand_name_list.append(brand_name)
        건강기능['brand_name'] = brand_name_list
        
    elif group == '생활용품':
        p_name = 생활용품['p_name']
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('(\((.*?)\))|(초특가)|(일시불)|(무이자)|(일\))|(무\))|(\[(.*?)\])|(1+1)|(ALL)|(파격가)|(1\+1)|(2019년)','', name)
            brand_name = re.sub('(1세트)|(완벽더블구성)|(국내제작)|(국내제조)|(D\))|(5세트)|(가\))|(KF94)|(기본구성)|(한세트)|(New)','', brand_name)

            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]

            brand_name = brand_name.replace('보국미니히터BKH-1083P+BKH-1083P','보국미니히터')
            brand_name = brand_name.replace('보국미니히터BKH-1083P','보국미니히터')
            brand_name = brand_name.replace('까사마루행거', '까사마루')


            brand_name = brand_name.replace('붙이는', '붙이는 따스림')
            brand_name_list.append(brand_name)
        생활용품['brand_name'] = brand_name_list
        
    elif group == '침구':
        p_name = 침구['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(일\))|(\(무\))|(일\))|(무\))','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name = brand_name.replace('리앤코리아', '리앤')
            brand_name_list.append(brand_name)
        침구['brand_name'] = brand_name_list
        
    elif group == '주방':
        p_name = 주방['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(일시불)|(무이자)|(\((.*?)\))|(19년)|(2019년형)|([0-9]세트)', '', name)
            brand_name = re.sub('(국내생산)|(국내제조)|(신제품)|(프랑스직수입)|(서장훈의)', '', brand_name)
            brand_name = re.sub('[\[\]]', " ", str(brand_name))
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]

            brand_name = brand_name.replace('쿠쿠전기밥솥', '쿠쿠')
            brand_name = brand_name.replace('쿠첸압력밥솥', '쿠첸')
            brand_name = brand_name.replace('쿠첸인버터전자레인지', '쿠첸')

            brand_name = brand_name.replace('해피콜IH양면팬', '해피콜')
            brand_name = brand_name.replace('해피콜엑슬림블렌더', '해피콜')  
            brand_name = brand_name.replace('휴롬퀵스퀴저', '휴롬')
            brand_name_list.append(brand_name)
        주방['brand_name'] = brand_name_list
        
    elif group == '가전':
        p_name = 가전['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.findall('(LG)|(삼성)|(대우전자)|(캐리어)|(딤채)', name)
            brand_name = re.sub('[\'\'\,\[\]\(\)]', '', str(brand_name))
            brand_name = brand_name.strip()
            brand_name = brand_name.replace('삼성    LG', 'LG')
            brand_name_list.append(brand_name)
        가전['brand_name'] = brand_name_list
        
    elif group == '가구':
        p_name = 가구['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(일\))|(\(무\))|(일\))|(무\))|(\[SET\])|(뉴)|(\(무\)[0-9]인용)|([0-9]인용)','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        가구['brand_name'] = brand_name_list
        
    elif group == '이미용':
        p_name = 이미용['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(무이자\))|(NEW프리미엄)|(\(일시불\))|(\[라쉬반\］)|(\[완벽더블\])|(초특가\))','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name = brand_name.replace('TS샴푸/트리트먼트(추가)', 'TS')
            brand_name = brand_name.replace('TS샴푸세트(추가)', 'TS')   
            brand_name_list.append(brand_name)
        이미용['brand_name'] = brand_name_list
        
    elif group == '농수축':
        p_name = 농수축['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(무이자\))|(NEW프리미엄)|(\(일시불\))|(\[라쉬반\))|(특등급)|(\[완벽더블\])|(초특가\)|(\(a4\))|(가격인하)|(19년)|(사용불가))','', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.replace('하림뼈없는양념닭발세트', '하림')
            brand_name = brand_name.replace('AAC한우소머리곰탕양곰탕세트', 'AAC')
            brand_name = brand_name.replace('맛있는', '맛있는제주')
            brand_name = brand_name.replace('맛있는', '맛있는제주')
            brand_name = brand_name.replace('더커진거창사과', '거창사과')
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        농수축['brand_name'] = brand_name_list
        
    elif group == '잡화':
        p_name = 잡화['p_name']
        brand_name_list = []
        for name in p_name:

            brand_name = re.sub('(\(무\))|(\(일\))|(무이자)|(일시불)|(2019)|(S/S)', '', name)
            brand_name = re.sub('[\[\]]', " ", brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        잡화['brand_name'] = brand_name_list
        
    elif group == '속옷':
        p_name = 속옷['p_name']
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('[\[\]\(\)］]', " ", name)
            brand_name = re.sub('(ARS10%)|(무이자)|(세일20%)|(일시불)|(완벽더블)|(초특가)', '', brand_name)
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        속옷['brand_name'] = brand_name_list
        
    elif group == '의류':
        p_name = 의류['p_name']
        brand_name_list = []
        for name in p_name:
            brand_name = re.sub('(화이트라벨)|(19 SS)|([0-9])|([\[\]]|(F/W)|(FW))', '', str(name))
            brand_name = brand_name.strip()
            brand_name = brand_name.split(' ')[0]
            brand_name_list.append(brand_name)
        의류['brand_name'] = brand_name_list
    
    else:
        break
        
p_group_name = [의류, 속옷, 농수축, 이미용, 가전, 가구, 침구, 생활용품, 잡화, 건강기능, 주방]

for c in p_group_name:
    #c_data =data[data['p_group'] == c]
    
    c_grouped = c.groupby('brand_name').sum()
    c_grouped['1분당 판매개수'] = c_grouped['sales'] / c_grouped['exp_min']
    c_dict = c_grouped[['1분당 판매개수']].sort_values('1분당 판매개수', ascending=False).to_dict()
    
    
    p_name = c['brand_name']
    c['brand_power'] = [c_dict['1분당 판매개수'][i] for i in p_name]

result_all = pd.concat([의류, 속옷, 농수축, 이미용, 가전, 가구, 침구, 생활용품, 잡화, 건강기능, 주방])
sum_df = result_all.sort_values('broadcast')
sum_df = sum_df.reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:180: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:147: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [ ]:
# 마스크, 택배의 검색량 feature 추가

client_id = "EDXLJguStAzjmnKluA9U"
client_secret = "u2Q9XCNqEy"
url = "https://openapi.naver.com/v1/datalab/search"
body = "{\"startDate\":\"2019-01-01\",\"endDate\":\"2020-07-01\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"마스크\",\"keywords\":[\"마스크\",\"공적마스크\",\"kf-94\",\"kf-80\",\"kf-ad\"]},{\"groupName\":\"택배\",\"keywords\":[\"택배\",\"택배조회\",\"cj대한통운\",\"우체국택배\"]}]}"

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    resp=response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

naver=json.loads(resp)

mask = naver['results'][0]
mask_data = mask['data']

mask_period_list = []
mask_ratio_list = []


for i in range(len(mask_data)):
    mask_period_list.append(mask_data[i]['period'])
    mask_ratio_list.append(mask_data[i]['ratio'])
    
    
mask_df = pd.DataFrame({'period' : mask_period_list, 'ratio' : mask_ratio_list})

parcel = naver['results'][-1]
parcel_data = parcel['data']

parcel_period_list = []
parcel_ratio_list = []

for i in range(len(parcel_data)):
    parcel_period_list.append(parcel_data[i]['period'])
    parcel_ratio_list.append(parcel_data[i]['ratio'])
    
parcel_df = pd.DataFrame({'period' : parcel_period_list, 'ratio' : parcel_ratio_list})

mask_data=mask['data']
mask_data_value=mask_data[0].values()

period_list=[]
ratio_list=[]


for i in range(len(mask_data)):
    period_list.append(mask_data[i]['period'])
    ratio_list.append(mask_data[i]['ratio'])

mask_df=pd.DataFrame({'period':period_list, 'ratio':ratio_list})

sum_df['YEAR_DAY'] = pd.to_datetime(sum_df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')

mask_df.columns = ['YEAR_DAY','mask_ratio']
mask_df['YEAR_DAY'] = pd.to_datetime(mask_df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')
sum_df = pd.merge(left=sum_df, right=mask_df, how='left', on=['YEAR_DAY'])

parcel_df.columns = ['YEAR_DAY','parcel_ratio']
parcel_df['YEAR_DAY'] = pd.to_datetime(parcel_df['YEAR_DAY'], format='%Y-%m-%d', errors='raise')
sum_df = pd.merge(left=sum_df, right=parcel_df, how='left', on=['YEAR_DAY'])

In [ ]:
sum_df.to_csv('c:/bc/sum_df.csv', encoding='utf-8-sig',index=False)